In [ ]:
# Install hoplite and TF 2.20
!pip install --upgrade pip
!pip install git+https://github.com/google-research/perch-hoplite.git
!pip install tensorflow~=2.20.0

In [ ]:
# Imports
import os
from matplotlib import pyplot as plt
import numpy as np
from etils import epath

# Hoplite imports
from perch_hoplite.agile import audio_loader
from perch_hoplite.agile import classifier
from perch_hoplite.agile import classifier_data
from perch_hoplite.agile import embedding_display
from perch_hoplite.agile import source_info
from perch_hoplite.db  import brutalism
from perch_hoplite.db import score_functions
from perch_hoplite.db  import search_results
from perch_hoplite.db import sqlite_usearch_impl
from perch_hoplite.zoo import model_configs

In [ ]:
# Copy hoplite database locally.
intaka_path = epath.Path('gs://chirp-public-bucket/soundscapes/intaka')

# Copy the hoplite database to the Colab local storage.
# The usearch.index file is large, so this may take a minute.
print("Copying database files...")
for fp in intaka_path.glob('hoplite*'):
  print(f"Copying {fp}...")
  with fp.open('rb') as f:
    with open(fp.name, 'wb') as g:
      g.write(f.read())

print("Copying search index (this may take a moment)...")
with (intaka_path / 'usearch.index').open('rb') as f:
  with open('usearch.index', 'wb') as g:
    g.write(f.read())

# Update the DB with override values.
# We point the db to the google cloud bucket containing the audio files.
db_path = '/content'
db = sqlite_usearch_impl.SQLiteUsearchDB.create(db_path)

# Update Model Config
model_cfg = db.get_metadata('model_config')
model_cfg.model_config.tfhub_path = 'google/bird-vocalization-classifier/tensorFlow2/perch_v2_cpu'
model_cfg.model_config.tfhub_version = 1
db.insert_metadata('model_config', model_cfg)

# Update Audio Sources Config
sources_cfg = db.get_metadata('audio_sources')
sources_cfg.audio_globs[0]['base_path'] = 'gs://chirp-public-bucket/soundscapes/intaka'
db.insert_metadata('audio_sources', sources_cfg)

db.commit()
print("SUCCESS: Database setup complete.")

In [ ]:
# Initialize Model and Audio Loader
annotator_id = 'linnaeus'  # Identifier for your labels

# 1. Get configuration from the Intaka DB
db_model_config = db.get_metadata('model_config')
embed_config = db.get_metadata('audio_sources')

# 2. Initialize the model
print("Initializing model...")
model_class = model_configs.get_model_class(db_model_config.model_key)
embedding_model = model_class.from_config(db_model_config.model_config)

# 3. Initialize the audio loader
audio_sources = source_info.AudioSources.from_config_dict(embed_config)
if hasattr(embedding_model, 'window_size_s'):
  window_size_s = embedding_model.window_size_s
else:
  window_size_s = 5.0

audio_filepath_loader = audio_loader.make_filepath_loader(
    audio_sources=audio_sources,
    window_size_s=window_size_s,
    sample_rate_hz=embedding_model.sample_rate,
)
print("SUCCESS: Model and Loader initialized.")

In [ ]:
# Load query audio.
# using Red-eyed Dove (xc746686) as requested for the Intaka session
query_uri = 'xc746686'
query_label = 'redeye'

print(f"Loading query: {query_uri} ({query_label})")
query = embedding_display.QueryDisplay(
    uri=query_uri, offset_s=0.0, window_size_s=5.0, sample_rate_hz=32000)
_ = query.display_interactive()

In [ ]:
#@title Display 50 Audio Results (Fixed)
# 1 Fix Imports
from perch_hoplite.agile import embedding_display
from perch_hoplite.db import search_results

# 2  Setup the query for Red-eyed Dove
query_label = 'redeye'
num_results = 50

print(f"Searching for {num_results} similar audio clips...")

# 3 Ensure we have the search results
query_embedding = embedding_model.embed(query.get_audio_window()).embeddings[0, 0]
ann_matches = db.ui.search(query_embedding, count=num_results)

results = search_results.TopKSearchResults(top_k=num_results)
for k, d in zip(ann_matches.keys, ann_matches.distances):
  results.update(search_results.SearchResult(k, d))

# 4. Create and Display the Audio Players
print("Generating audio players... (This might take a few seconds)")
display_results = embedding_display.EmbeddingDisplayGroup.from_search_results(
    results, db, sample_rate_hz=32000, frame_rate=100,
    audio_loader=audio_filepath_loader)

# 5. Show the widgets
print("READY: Please listen and click 'redeye' on the matching birds below.")
display_results.display(positive_labels=[query_label])

In [ ]:
#@title 1. Save Data Labels
# Define your name/ID
annotator_id = 'linnaeus'

prev_lbls, new_lbls = 0, 0
print("Harvesting your labels...")

# This grabs the 'green' clicks from the display you just used
try:
    for lbl in display_results.harvest_labels(annotator_id):
        check = db.insert_label(lbl, skip_duplicates=True)
        new_lbls += check
        prev_lbls += (1 - check)

    print(f'\nSUCCESS: New labels saved: {new_lbls}')
    print(f'Previous labels (duplicates skipped): {prev_lbls}')

    if new_lbls == 0 and prev_lbls == 0:
        print("WARNING: No labels were saved. Did you click the 'redeye' buttons?")

except NameError:
    print("ERROR: Please run the 'Display Results' cell first.")

In [ ]:
#@title 2. Train Classifier
from perch_hoplite.agile import classifier
from perch_hoplite.agile import classifier_data
import numpy as np

# Classifier settings
learning_rate = 1e-3
num_steps = 128
batch_size = 128

print("Preparing training data")
# Prepare the data
data_manager = classifier_data.AgileDataManager(
    target_labels=None, # Auto-detects 'redeye' from your saved labels
    db=db,
    train_ratio=0.9,
    min_eval_examples=1,
    batch_size=batch_size,
    weak_negatives_batch_size=batch_size,
    rng=np.random.default_rng(seed=5))

print(f'Training for labels: {data_manager.get_target_labels()}')

# Train the model
linear_classifier, eval_scores = classifier.train_linear_classifier(
    data_manager=data_manager,
    learning_rate=learning_rate,
    weak_neg_weight=0.05,
    num_train_steps=num_steps,
)

print('\n' + '-' * 80)
print(f'Top-1 Accuracy:    {eval_scores["top1_acc"]:.3f}')
print(f'ROC AUC Score:     {eval_scores["roc_auc"]:.3f}')
linear_classifier.save(os.path.join(db_path, 'agile_classifier_v2.pt'))

In [ ]:
#@title 3. Export Labeled Segments to CSV
import pandas as pd
from google.colab import files

data_records = []

# Gather all labels from the database
print("Gathering data for CSV")
for cls in db.get_classes():
  idxes = db.get_embeddings_by_label(cls)
  for idx in idxes:
    s = db.get_embedding_source(idx)
    for lbl in db.get_labels(idx):
      data_records.append({
          'source_id': s.source_id,
          'offset_s': s.offsets[0],
          'label': lbl.label,
          'type': lbl.type,
          'annotator': annotator_id
      })

# Create CSV and download
if data_records:
    df = pd.DataFrame(data_records)
    csv_filename = 'intaka_labeled_segments.csv'
    df.to_csv(csv_filename, index=False)
    print(f"\nSUCCESS: Saved {len(df)} labels to {csv_filename}")

    # Auto-download the file
    files.download(csv_filename)
else:
    print("\nNo labels found! Please go back to Step 1.")

In [ ]:
#@title Search for Bird #2: Laughing Dove
from perch_hoplite.agile import embedding_display
from perch_hoplite.db import search_results

# 1. Setup for bird 2 (Laughing Dove)
query_uri = 'xc666993'   # Id for Laughing Dove
query_label = 'laudov'   # Label for Laughing Dove

print(f"Loading query: {query_uri} ({query_label})")
query = embedding_display.QueryDisplay(
    uri=query_uri, offset_s=0.0, window_size_s=5.0, sample_rate_hz=32000)
_ = query.display_interactive()

# 2. SEARCH
print(f"Searching for 50 matches for {query_label}...")
query_embedding = embedding_model.embed(query.get_audio_window()).embeddings[0, 0]
ann_matches = db.ui.search(query_embedding, count=50)

results = search_results.TopKSearchResults(top_k=50)
for k, d in zip(ann_matches.keys, ann_matches.distances):
  results.update(search_results.SearchResult(k, d))

# 3. DISPLAY PLAYERS
print("READY: Listen and click 'laudov' for the correct matches below.")
display_results = embedding_display.EmbeddingDisplayGroup.from_search_results(
    results, db, sample_rate_hz=32000, frame_rate=100,
    audio_loader=audio_filepath_loader)
display_results.display(positive_labels=[query_label])

In [ ]:
#@title Save Labels for Bird 2
annotator_id = 'linnaeus'

prev_lbls, new_lbls = 0, 0
print(f"Harvesting labels for {query_label}...")

try:
    for lbl in display_results.harvest_labels(annotator_id):
        check = db.insert_label(lbl, skip_duplicates=True)
        new_lbls += check
        prev_lbls += (1 - check)

    print(f'\nSUCCESS: Added {new_lbls} new labels.')

    # Recalculate total labels by iterating through the db's public methods
    all_current_labels = []
    for cls_name in db.get_classes():
        idxes_for_class = db.get_embeddings_by_label(cls_name)
        for idx in idxes_for_class:
            labels_for_embedding = db.get_labels(idx)
            all_current_labels.extend(labels_for_embedding)

    total_labels_count = len(all_current_labels)
    print(f'Total labels in database (Birds 1 & 2): {total_labels_count}')

except NameError:
    print("ERROR: Please run the 'Search for Bird #2' cell first.")

In [ ]:
#@title 2. Train Classifier
from perch_hoplite.agile import classifier
from perch_hoplite.agile import classifier_data
import numpy as np

# Classifier settings
learning_rate = 1e-3
num_steps = 128
batch_size = 128

print("Preparing training data...")
# Prepare the data
data_manager = classifier_data.AgileDataManager(
    target_labels=None, # Auto-detects 'redeye' from your saved labels
    db=db,
    train_ratio=0.9,
    min_eval_examples=1,
    batch_size=batch_size,
    weak_negatives_batch_size=batch_size,
    rng=np.random.default_rng(seed=5))

print(f'Training for labels: {data_manager.get_target_labels()}')

# Train the model
linear_classifier, eval_scores = classifier.train_linear_classifier(
    data_manager=data_manager,
    learning_rate=learning_rate,
    weak_neg_weight=0.05,
    num_train_steps=num_steps,
)

print('\n' + '-' * 80)
print(f'Top-1 Accuracy:    {eval_scores["top1_acc"]:.3f}')
print(f'ROC AUC Score:     {eval_scores["roc_auc"]:.3f}')

# Save the trained model locally
linear_classifier.save(os.path.join(db_path, 'agile_classifier_v2.pt'))

In [ ]:
#@title Train & Export Final CSV
import pandas as pd
from google.colab import files
from perch_hoplite.agile import classifier_data, classifier

# 1. TRAIN (Detects both 'redeye' and 'laudov')
print("Training classifier on ALL birds...")
data_manager = classifier_data.AgileDataManager(
    target_labels=None,
    db=db,
    train_ratio=0.9,
    min_eval_examples=1,
    batch_size=128,
    weak_negatives_batch_size=128,
    rng=np.random.default_rng(seed=5))

print(f"Found targets: {data_manager.get_target_labels()}") # Should show both birds

linear_classifier, eval_scores = classifier.train_linear_classifier(
    data_manager=data_manager,
    learning_rate=1e-3,
    weak_neg_weight=0.05,
    num_train_steps=128,
)
print(f"Final Accuracy: {eval_scores['top1_acc']:.3f}\n")

# 2. EXPORT CSV
print("Creating Final CSV...")
data_records = []
for cls in db.get_classes():
  idxes = db.get_embeddings_by_label(cls)
  for idx in idxes:
    s = db.get_embedding_source(idx)
    for lbl in db.get_labels(idx):
      data_records.append({
          'source_id': s.source_id,
          'offset_s': s.offsets[0],
          'label': lbl.label,
          'type': lbl.type,
          'annotator': annotator_id
      })

if data_records:
    df = pd.DataFrame(data_records)
    csv_filename = 'intaka_labeled_segments_final.csv'
    df.to_csv(csv_filename, index=False)
    print(f"SUCCESS: Saved {len(df)} labels to {csv_filename}")
    files.download(csv_filename)
else:
    print("No labels found.")